In [2]:
import numpy as np
import pandas as pd
!pip install emoji
import emoji
import joblib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, SimpleRNN, Embedding

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

2022-08-19 19:22:36.057351: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-19 19:22:36.057380: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
data = pd.read_csv('emoji_data.csv', header = None)
data

,0,1
0,French macaroon is so tasty,4
1,work is horrible,3
2,I am upset,3
3,throw the ball,1
4,Good joke,2
...,...,...
178,lets brunch some day,4
179,dance with me,2
180,she is a bully,3
181,she plays baseball,1


In [4]:
#Dictionary to MAP integers to emoji

emoji_dict = {
    0: ":red_heart:",
    1: ":baseball:",
    2: ":grinning_face_with_big_eyes:",
    3: ":disappointed_face:",
    4: ":fork_and_knife_with_plate:"
}

def label_to_emoji(label):
    return emoji.emojize(emoji_dict[label])

In [5]:
emoji.emojize(emoji_dict[0])

'❤️'

In [6]:
X = data[0].values
Y = data[1].values

# Embeddings

In [7]:
file = open('glove.6B.100d.txt', 'r', encoding = 'utf8')
content = file.readlines()
file.close()

# content

In [8]:
embeddings = {}

for line in content:
    line = line.split()
    embeddings[line[0]] = np.array(line[1:], dtype = float)

In [9]:
def get_maxlen(data):
    maxlen = 0
    for sent in data:
        maxlen = max(maxlen, len(sent))
    return maxlen


In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
word2index = tokenizer.word_index

In [11]:
Xtokens = tokenizer.texts_to_sequences(X)

maxlen = get_maxlen(Xtokens)
print(maxlen)
#All tokens must be of same length so padding is done
Xtrain = pad_sequences(Xtokens, maxlen = maxlen,  padding = 'post', truncating = 'post')
Xtrain

10


array([[103, 104,   3, ...,   0,   0,   0],
       [106,   3, 107, ...,   0,   0,   0],
       [  1,   7, 108, ...,   0,   0,   0],
       ...,
       [ 14,   3,   5, ...,   0,   0,   0],
       [ 14, 310,  26, ...,   0,   0,   0],
       [  1,  24,  22, ...,   0,   0,   0]], dtype=int32)

In [12]:
Y

array([4, 3, 3, 1, 2, 1, 4, 3, 4, 1, 3, 3, 2, 2, 4, 3, 2, 3, 3, 1, 3, 2,
       2, 2, 0, 1, 0, 4, 2, 0, 2, 0, 0, 3, 4, 0, 2, 1, 3, 1, 0, 4, 0, 3,
       0, 4, 2, 3, 4, 2, 2, 3, 0, 2, 2, 3, 2, 3, 2, 2, 3, 3, 0, 2, 3, 0,
       2, 0, 0, 2, 3, 2, 4, 1, 3, 3, 0, 0, 3, 2, 0, 3, 0, 2, 2, 4, 2, 2,
       0, 0, 2, 3, 0, 4, 2, 1, 2, 3, 3, 2, 3, 0, 3, 0, 2, 0, 2, 3, 4, 3,
       1, 3, 4, 3, 2, 3, 3, 3, 1, 4, 4, 2, 2, 1, 1, 2, 3, 2, 3, 4, 2, 3,
       0, 2, 0, 0, 4, 3, 4, 2, 3, 2, 3, 4, 2, 1, 2, 4, 3, 1, 3, 2, 3, 2,
       2, 3, 3, 2, 4, 0, 0, 0, 3, 0, 0, 1, 1, 2, 2, 2, 0, 3, 2, 3, 3, 1,
       2, 2, 4, 2, 3, 1, 2])

In [13]:
Ytrain = to_categorical(Y)


# Model

In [14]:
embed_size = 100
embedding_matrix = np.zeros((len(word2index)+1, embed_size))

for word, i in word2index.items():
    embed_vector = embeddings[word]
    embedding_matrix[i] = embed_vector

In [15]:
embedding_matrix

array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [-0.046539,  0.61966 ,  0.56647 , ..., -0.37616 , -0.032502,
         0.8062  ],
       [-0.49886 ,  0.76602 ,  0.89751 , ..., -0.41179 ,  0.40539 ,
         0.78504 ],
       ...,
       [-0.46263 ,  0.069864,  0.69095 , ..., -0.29174 ,  0.32041 ,
         0.21202 ],
       [ 0.073242,  0.11134 ,  0.62281 , ...,  0.53417 , -0.1646  ,
        -0.27516 ],
       [ 0.29019 ,  0.80497 ,  0.31187 , ..., -0.33603 ,  0.45998 ,
        -0.11278 ]])

In [16]:
model = Sequential([
    Embedding(input_dim = len(word2index) + 1,
              output_dim = embed_size,
              input_length = maxlen,
              weights = [embedding_matrix],
              trainable = False
             ),
    
    LSTM(units = 20, return_sequences = True),
    LSTM(units = 4),
    Dense(5, activation = 'softmax')
])

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

2022-08-19 19:22:47.746631: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-19 19:22:47.746680: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-19 19:22:47.746732: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ggn000720): /proc/driver/nvidia/version does not exist
2022-08-19 19:22:47.747101: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
model.fit(Xtrain, Ytrain, epochs = 100)

Epoch 1/100
6/6 [==============================] - 4s 14ms/step - loss: 1.5852 - accuracy: 0.3005
Epoch 2/100
6/6 [==============================] - 0s 12ms/step - loss: 1.5561 - accuracy: 0.3607
Epoch 3/100
6/6 [==============================] - 0s 12ms/step - loss: 1.5332 - accuracy: 0.3661
Epoch 4/100
6/6 [==============================] - 0s 11ms/step - loss: 1.5128 - accuracy: 0.4098
Epoch 5/100
6/6 [==============================] - 0s 12ms/step - loss: 1.4928 - accuracy: 0.4372
Epoch 6/100
6/6 [==============================] - 0s 11ms/step - loss: 1.4705 - accuracy: 0.4481
Epoch 7/100
6/6 [==============================] - 0s 11ms/step - loss: 1.4385 - accuracy: 0.4863
Epoch 8/100
6/6 [==============================] - 0s 14ms/step - loss: 1.4070 - accuracy: 0.4973
Epoch 9/100
6/6 [==============================] - 0s 14ms/step - loss: 1.3652 - accuracy: 0.5027
Epoch 10/100
6/6 [==============================] - 0s 12ms/step - loss: 1.3292 - accuracy: 0.5027
Epoch 11/100
6/6 [=

6/6 [==============================] - 0s 11ms/step - loss: 0.3099 - accuracy: 0.8852
Epoch 84/100
6/6 [==============================] - 0s 13ms/step - loss: 0.3073 - accuracy: 0.9071
Epoch 85/100
6/6 [==============================] - 0s 11ms/step - loss: 0.3049 - accuracy: 0.9180
Epoch 86/100
6/6 [==============================] - 0s 11ms/step - loss: 0.3019 - accuracy: 0.9016
Epoch 87/100
6/6 [==============================] - 0s 10ms/step - loss: 0.2994 - accuracy: 0.9071
Epoch 88/100
6/6 [==============================] - 0s 10ms/step - loss: 0.2930 - accuracy: 0.9180
Epoch 89/100
6/6 [==============================] - 0s 10ms/step - loss: 0.2904 - accuracy: 0.9071
Epoch 90/100
6/6 [==============================] - 0s 13ms/step - loss: 0.2872 - accuracy: 0.9016
Epoch 91/100
6/6 [==============================] - 0s 12ms/step - loss: 0.2853 - accuracy: 0.8907
Epoch 92/100
6/6 [==============================] - 0s 13ms/step - loss: 0.2830 - accuracy: 0.8907
Epoch 93/100
6/6 [=====

In [18]:

test_seq = tokenizer.texts_to_sequences("i eat food")
Xtest = pad_sequences(test_seq, maxlen = maxlen, padding = 'post', truncating = 'post')

y_pred = model.predict(Xtest)
print(y_pred)
y_pred1 = np.argsort(y_pred, axis = 1)
labels = list(y_pred1[0][-2:])
print(list(y_pred1[0][-2:]))
for i in labels:
    print(label_to_emoji(i))

1/1 [==============================] - 1s 1s/step
[[0.01749783 0.01419746 0.44008014 0.5129961  0.01522854]
 [0.00212061 0.01984117 0.9352627  0.02559329 0.01718217]
 [0.00212061 0.01984117 0.9352627  0.02559329 0.01718217]
 [0.00204744 0.01584689 0.9343629  0.03388417 0.01385871]
 [0.00212061 0.01984117 0.9352627  0.02559329 0.01718217]
 [0.00212061 0.01984117 0.9352627  0.02559329 0.01718217]
 [0.00212061 0.01984117 0.9352627  0.02559329 0.01718217]
 [0.00212061 0.01984117 0.9352627  0.02559329 0.01718217]
 [0.00212061 0.01984117 0.93526274 0.02559329 0.01718217]
 [0.00212061 0.01984117 0.93526274 0.02559329 0.01718217]]
[2, 3]
😃
😞


In [19]:
def save(model):
    # Save the trained weights
    model.save_weights('models/model_weights.h5')

    # Save the model architecture
    with open('models/model_architecture.json', 'w') as f:
        f.write(model.to_json())

    # Save the tokenizer
    with open('models/tokenizer.json', 'w') as f:
        f.write(tokenizer.to_json())

In [20]:
save(model)